<a href="https://colab.research.google.com/github/GabrielG1997/COLAB/blob/main/NaturalLanguageProcessingExercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description

This exercise was the last one of the Bootcamp [BairesDev - Machine Learning Practitioner](https://web.dio.me/track/coding-the-future-baires-dev-machine-learning-practitioner) provided by [DIO](https://web.dio.me/) and the main objective was to develop a virtual assistant.

* The virtual assistant should use NLP to listen the user input and convert it to text.

* Based on the user input, the virtual assistant should select one of the available options and generate an speech to the user as an output

> Colab can't run code that access directly the resources of the local machine like the microphone or camera, to conter-measure this we created a JS script that uses the browser to generate an audio file, this file them is downloaded into the local machine and uploaded in the Colab project, after that its used to process the input and generate the expected output.



In [94]:
!pip install SpeechRecognition
!pip install gTTS
!pip install playsound
!pip install pyjokes
!pip install wikipedia
!pip install pydub ffmpeg #audio format conversion to WAV PCM
import speech_recognition as sr
from pydub import AudioSegment
from gtts import gTTS
import os
import tempfile
import datetime
import playsound
import pyjokes
import wikipedia
import webbrowser
from IPython.display import Javascript, display, Audio, HTML
from google.colab import files


In [129]:
from IPython.display import Javascript, display
from google.colab import files

# JS Code to generate /download Audio
js_code = """
const recorderButton = document.createElement("button");
recorderButton.innerHTML = "Click to Record";
recorderButton.style.display = "block";
recorderButton.style.margin = "10px auto";
document.body.appendChild(recorderButton);

const stopButton = document.createElement("button");
stopButton.innerHTML = "Stop Recording";
stopButton.style.display = "block";
stopButton.style.margin = "10px auto";
stopButton.style.visibility = "hidden";
document.body.appendChild(stopButton);

let chunks = [];
let recorder;
let audioBlob;

recorderButton.onclick = async () => {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    recorder = new MediaRecorder(stream);
    recorder.ondataavailable = (e) => chunks.push(e.data);
    recorder.onstop = () => {
        audioBlob = new Blob(chunks, { type: "audio/wav" });
        const url = URL.createObjectURL(audioBlob);
        const anchor = document.createElement("a");
        anchor.style.display = "block";
        anchor.style.margin = "10px auto";
        anchor.href = url;
        anchor.download = "recorded_audio.wav";
        anchor.innerHTML = "Download Audio";
        document.body.appendChild(anchor);
    };

    recorder.start();
    recorderButton.style.visibility = "hidden";
    stopButton.style.visibility = "visible";
};

stopButton.onclick = () => {
    recorder.stop();
    recorderButton.style.visibility = "visible";
    stopButton.style.visibility = "hidden";
};
"""

# Render JS block on Colab
display(Javascript(js_code))


<IPython.core.display.Javascript object>

In [130]:
uploaded = files.upload()  # upload file "recorded_audio.wav"
audio_file = list(uploaded.keys())[0]

print(f"Audio Uploaded: {audio_file}")

Saving recorded_audio.wav to recorded_audio.wav
Audio Uploaded: recorded_audio.wav


In [131]:
# Load and convert audio to WAV PCM
def convert_audio_to_wav(input_file, output_file):
    try:
        audio = AudioSegment.from_file(input_file)
        audio = audio.set_frame_rate(16000).set_channels(1)  # Taxa de amostragem e canais
        audio.export(output_file, format="wav")
        print(f"File converted to: {output_file}")
    except Exception as e:
        print(f"Error converting file: {e}")


input_file = "recorded_audio.wav"
output_file = "converted_audio.wav"
convert_audio_to_wav(input_file, output_file)

File converted to: converted_audio.wav


In [132]:
# get mic audio
def get_audio():
  recognizer = sr.Recognizer()
  audio_path = "converted_audio.wav"

  with sr.AudioFile(audio_path) as source:
    audio = recognizer.record(source)
    try:
      print("Recognized text:", recognizer.recognize_google(audio))
    except sr.UnknownValueError:
      print("Could not understand audio.")
    except sr.RequestError as e:
      print(f"API ERROR: {e}")
    return recognizer.recognize_google(audio)

In [133]:
# speak convert audio to text
def speak(text):
  with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as temp_audio:
    tts = gTTS(text=text, lang='en')
    tts.save(temp_audio.name)
  return Audio(temp_audio.name, autoplay=True)

In [134]:
# executing
print("I am listening...")
text = get_audio()
message = ""

if 'YouTube' in text:
  print("Opening Youtube...")
  message = "Opening Youtube"
  # url = f"https://www.youtube.com" -> this won't work on COLAB environment
  # webbrowser.open(url) -> this won't work on COLAB environment
  display(HTML("<a href='https://www.youtube.com' target='_blank'>Click here to open YouTube</a>"))

elif 'search' in text:
  print("Searching Wikipedia...")
  #speak("Searching Wikipedia...")
  query = text.replace("search", "")
  result = wikipedia.summary(query, sentences = 3)
  #speak("According to Wikipedia")
  print(result)
  #speak(result)
  message = result


elif 'joke' in text:
  print("Getting a joke...")
  #speak(pyjokes.get_joke())
  message = pyjokes.get_joke()

elif 'exit' in text:
  print("Goodbye, till next time...")
  #speak("Goodbye, till next time")
  message = "Goodbye, till next time"

speak(message)


I am listening...
Recognized text: search Wikipedia
Searching Wikipedia...
The Wikimedia Foundation, Inc., abbreviated WMF, is an American 501(c)(3) nonprofit organization headquartered in San Francisco, California, and registered there as a charitable foundation. It is the host of Wikipedia, the seventh most visited website in the world. It also hosts fourteen related open collaboration projects, and supports the development of MediaWiki, the wiki software which underpins them all.
